In [ ]:
import gradio as gr
import google.generativeai as genai
from gtts import gTTS
import tempfile


genai.configure(api_key="API_KEY")
model = genai.GenerativeModel('gemini-1.5-flash')


interview_state = {
    "questions": [],
    "answers": [],
    "current_question": "",
    "question_index": 0
}

def start_interview(skills_text):
    interview_state["questions"] = []
    interview_state["answers"] = []
    interview_state["question_index"] = 0

    prompt = (
    f"You are an AI interviewer. Based on these skills: {skills_text}, generate exactly 5 interview questions.\n"
    "Respond ONLY with the questions, each question on its own line.\n"
    "Do NOT add any introduction, numbering, or extra text."
    )

    response = model.generate_content(prompt)
    questions = response.text.strip().split('\n')
    interview_state["questions"] = [q for q in questions if q.strip() != '']
    
    interview_state["current_question"] = interview_state["questions"][0]
    return f"First Question: {interview_state['current_question']}"

def next_question(user_answer):
    interview_state["answers"].append(user_answer)
    interview_state["question_index"] += 1
    
    if interview_state["question_index"] < len(interview_state["questions"]):
        interview_state["current_question"] = interview_state["questions"][interview_state["question_index"]]
        return f"Next Question: {interview_state['current_question']}"
    else:
        return "Interview complete. Click 'Get Feedback'!"

def get_feedback():
    qa_pairs = "\n".join([f"Q: {q}\nA: {a}" for q, a in zip(interview_state["questions"], interview_state["answers"])])
    prompt = f"You are an AI interviewer. Here is the interview:\n{qa_pairs}\nGive feedback about mistakes and final result in simple terms."
    
    response = model.generate_content(prompt)
    feedback_text = response.text.strip()
    
    
    tts = gTTS(text=feedback_text)
    temp_audio = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
    tts.save(temp_audio.name)
    
    return feedback_text, temp_audio.name


with gr.Blocks() as demo:
    gr.Markdown("## AI Interviewer (Skills-based)")
    
    skills_input = gr.Textbox(label="Enter your skills")
    start_btn = gr.Button("Start Interview")
    question_output = gr.Textbox(label="Interview Question")
    
    answer_input = gr.Textbox(label="Your Answer")
    next_btn = gr.Button("Next Question")
    
    feedback_btn = gr.Button("Get Feedback & Speech")
    feedback_output = gr.Textbox(label="Feedback")
    audio_output = gr.Audio(label="Feedback Audio")
    
    
    start_btn.click(start_interview, inputs=skills_input, outputs=question_output)
    next_btn.click(next_question, inputs=answer_input, outputs=question_output)
    feedback_btn.click(get_feedback, inputs=[], outputs=[feedback_output, audio_output])


demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
